In [ ]:
from gatenlp import Document
from gatenlp.corpora import ListCorpus
import json
import os

In [ ]:
from gatenlp.visualization import CorpusViewer

corpus = loadCorpus()
viewer = CorpusViewer(corpus)
viewer.show()

In [ ]:
create_gold_standard_json()